# For cleaning each city individually

In [1]:
import pandas as pd
import gzip
import shutil

In [22]:
from pathlib import Path
cwd = Path('cleaning.py').parents[0]

In [23]:
cwd

PosixPath('.')

In [26]:
with gzip.open('{}/listings.csv.gz'.format(cwd), 'rb') as f_in:
    with open('{}/listings2.csv'.format(cwd), 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [25]:
citylistingsdf = pd.read_csv('listings.csv')

In [5]:
citylistingsdf2 = pd.read_csv('listings2.csv')

In [6]:
listingsdfcleaned = citylistingsdf[['id','name','host_id','neighbourhood','latitude','longitude','room_type','price','availability_365']]

In [7]:
listings2dfcleaned = citylistingsdf2[['id','listing_url','name','neighbourhood_cleansed','latitude','longitude','property_type','room_type','accommodates','availability_30','bathrooms_text','bedrooms','beds']]

In [8]:
listings_comb = pd.merge(listingsdfcleaned,listings2dfcleaned,on=['id','name','latitude','longitude','room_type'],how='outer')

In [9]:
listings_comb = listings_comb.dropna()

In [10]:
listings_comb = listings_comb.drop(['neighbourhood_cleansed'],axis=1)

In [11]:
listings_comb['neighbourhood'] = listings_comb['neighbourhood'].astype('category')
listings_comb['room_type'] = listings_comb['room_type'].astype('category')
listings_comb['property_type'] = listings_comb['property_type'].astype('category')

In [12]:
listings_comb['bedrooms'] = listings_comb['bedrooms'].astype('int')
listings_comb['beds'] = listings_comb['beds'].astype('int')
listings_comb['accommodates'] = listings_comb['accommodates'].astype('int')

In [13]:
import re

In [14]:
def get_bath(str):
    x = re.search('^\d\S*\S*\S*',str)
    if x:
        return x.group()

In [15]:
listings_comb['bathrooms_text'] = listings_comb['bathrooms_text'].apply(get_bath)

In [16]:
listings_comb['bathrooms_text'] = listings_comb['bathrooms_text'].astype('float')

In [19]:
listings_comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1939 entries, 0 to 2098
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id                1939 non-null   int64   
 1   name              1939 non-null   object  
 2   host_id           1939 non-null   float64 
 3   neighbourhood     1939 non-null   category
 4   latitude          1939 non-null   float64 
 5   longitude         1939 non-null   float64 
 6   room_type         1939 non-null   category
 7   price             1939 non-null   float64 
 8   availability_365  1939 non-null   float64 
 9   listing_url       1939 non-null   object  
 10  property_type     1939 non-null   category
 11  accommodates      1939 non-null   int64   
 12  availability_30   1939 non-null   float64 
 13  bathrooms_text    1938 non-null   float64 
 14  bedrooms          1939 non-null   int64   
 15  beds              1939 non-null   int64   
dtypes: category(3), float64(

In [18]:
listings_comb.to_csv('city_listings_comb.csv')

In [20]:
listings_comb.to_pickle('city_listings_comb.pkl')